In [ ]:
!pip install ultralytics

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.208 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 29.3/166.8 GB disk)


In [ ]:
!pip install wandb==0.15.0
import os
import cv2
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import shutil
from shutil import copyfile
import matplotlib.pyplot as plt
from matplotlib import animation, rc
rc('animation', html='jshtml')
from PIL import Image
import ultralytics
from ultralytics import YOLO

  Using cached wandb-0.15.0-py3-none-any.whl (2.0 MB)


Download Kaggle drone dataset UAV (https://www.kaggle.com/datasets/dasmehdixtr/drone-dataset-uav/data) and upload to notebook

In [ ]:
!unzip drone_UAV.zip

Archive:  drone_UAV.zip
  inflating: dataset_xml_format/dataset_xml_format/foto00088.png  
  inflating: dataset_xml_format/dataset_xml_format/foto00088.xml  
  inflating: dataset_xml_format/dataset_xml_format/foto00117.png  
  inflating: dataset_xml_format/dataset_xml_format/foto00117.xml  
  inflating: dataset_xml_format/dataset_xml_format/foto00204.png  
  inflating: dataset_xml_format/dataset_xml_format/foto00204.xml  
  inflating: dataset_xml_format/dataset_xml_format/foto00233.png  
  inflating: dataset_xml_format/dataset_xml_format/foto00233.xml  
  inflating: dataset_xml_format/dataset_xml_format/foto00262.png  
  inflating: dataset_xml_format/dataset_xml_format/foto00262.xml  
  inflating: dataset_xml_format/dataset_xml_format/foto00291.png  
  inflating: dataset_xml_format/dataset_xml_format/foto00291.xml  
  inflating: dataset_xml_format/dataset_xml_format/foto00320.png  
  inflating: dataset_xml_format/dataset_xml_format/foto00320.xml  
  inflating: dataset_xml_format/datase

In [ ]:
if not(os.path.isdir('datasets')):
  !mkdir datasets
if not(os.path.isdir('datasets/train')):
  !mkdir datasets/train
if not(os.path.isdir('datasets/valid')):
  !mkdir datasets/valid

train_path='datasets/train/'
valid_path='datasets/valid/'

os.chdir("/content")
ano_paths=[]
for dirname, _, filenames in os.walk('drone_dataset_yolo/dataset_txt'):
    for filename in filenames:
        if filename[-4:]=='.txt':
            ano_paths+=[(os.path.join(dirname, filename))]

n=len(ano_paths)
print(n)
N=list(range(n))
random.shuffle(N)

train_ratio = 0.8
valid_ratio = 0.2


train_size = int(train_ratio*n)
valid_size = int(valid_ratio*n)

train_i = N[:train_size]
valid_i = N[train_size:train_size+valid_size]


print(train_i)
print(valid_i)

1360
[406, 1196, 671, 50, 222, 916, 1255, 1021, 1124, 785, 828, 1356, 257, 627, 1, 290, 979, 153, 1086, 1253, 75, 807, 929, 429, 849, 59, 1133, 918, 841, 1171, 729, 265, 255, 1283, 1017, 182, 933, 655, 355, 473, 863, 366, 1345, 1160, 856, 699, 34, 1078, 980, 440, 1230, 1173, 866, 18, 501, 1100, 1217, 608, 324, 546, 168, 1036, 468, 782, 465, 797, 1309, 1102, 741, 1341, 1166, 235, 174, 811, 1249, 926, 888, 974, 141, 1346, 330, 911, 764, 1336, 3, 256, 356, 1304, 657, 731, 684, 994, 1134, 503, 313, 1296, 1047, 832, 387, 992, 449, 80, 133, 844, 955, 651, 1129, 1198, 164, 1215, 818, 95, 1348, 260, 988, 1337, 1008, 1041, 311, 584, 1330, 377, 523, 695, 815, 624, 1128, 1225, 1233, 579, 1327, 708, 1300, 1241, 282, 750, 712, 263, 293, 474, 507, 1050, 29, 549, 346, 433, 280, 561, 16, 27, 650, 130, 85, 454, 108, 460, 435, 497, 878, 25, 953, 1069, 567, 364, 827, 1301, 744, 388, 1019, 810, 367, 1203, 545, 121, 172, 8, 808, 1316, 687, 135, 1270, 944, 1095, 51, 760, 940, 721, 487, 398, 574, 659, 765, 5

In [ ]:
for i in train_i:
    ano_path=ano_paths[i]
    img_path=os.path.join('drone_dataset_yolo/dataset_txt',
                          ano_path.split('/')[-1][0:-4]+'.jpg')
    try:
        !cp {ano_path} {train_path}
        !cp {img_path} {train_path}
    except:
        continue
print(len(os.listdir(train_path)))

cp: cannot stat 'drone_dataset_yolo/dataset_txt/classes.jpg': No such file or directory
2175


In [ ]:
for i in valid_i:
    ano_path=ano_paths[i]
    img_path=os.path.join('drone_dataset_yolo/dataset_txt',
                          ano_path.split('/')[-1][0:-4]+'.jpg')
    try:
        !cp {ano_path} {valid_path}
        !cp {img_path} {valid_path}
    except:
        continue
print(len(os.listdir(valid_path)))

544


In [ ]:
import yaml

data_yaml = dict(
    train ='train',
    val ='valid',
    test='test',
    nc =1,
    names =['drone']
)

with open('data.yaml', 'w') as outfile:
    yaml.dump(data_yaml, outfile, default_flow_style=True)

%cat data.yaml

{names: [drone], nc: 1, test: test, train: train, val: valid}


In [ ]:
names =['drone']
M=list(range(len(names)))
class_map=dict(zip(M,names))

In [ ]:
model = YOLO("yolov8x.pt")

100%|██████████| 131M/131M [00:00<00:00, 474MB/s] 



In [ ]:
!yolo task=detect mode=train model=yolov8x.pt data=data.yaml epochs=12 imgsz=480

Ultralytics YOLOv8.0.20 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8x.yaml, data=data.yaml, epochs=12, patience=50, batch=16, imgsz=480, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=False, dnn=False, plots=False, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset

In [ ]:
os.chdir("/content")
best_path = 'runs/detect/train/weights/best.pt'
model2 = YOLO(best_path)

In [ ]:
!pip install pytube
from pytube import YouTube

if not(os.path.isdir('raw_videos')):
  !mkdir raw_videos


links = ["https://www.youtube.com/watch?v=DhmZ6W1UAv4", "https://www.youtube.com/watch?v=YrydHPwRelI"]

for l in links:
    vid = YouTube(l)
    vid.streams.filter(progressive="True").get_highest_resolution().download("/content/raw_videos")

In [ ]:
vidpath = "/raw_videos"
os.chdir("/content")
names = os.listdir(vidpath)
titles = []
for name in names:
	if(name.endswith('.mp4')):
		titles.append(name)

#make directories
if not(os.path.isdir('detections')):
  !mkdir detections
for title in titles:
  path = os.path.join('detections', title)
  if not(os.path.isdir(path)):
    os.mkdir(path)


In [ ]:
titles

['Drone tracking 2.mp4', 'Drone Tracking 1.mp4']

In [13]:
os.listdir('/content/detections')

['Drone tracking 2.mp4', 'Drone Tracking 1.mp4']

In [19]:
os.chdir("/content")
for title in titles:
  video_path = os.path.join("/content/raw_videos", title)
  cap = cv2.VideoCapture(video_path)
  currentFrame = 0
  # Loop through the video frames
  while cap.isOpened():
      # Read a frame from the video
      success, frame = cap.read()

      if success:
          # Run YOLOv8 inference on the frame
          results = model2(frame)

          # Save result if detection found
          if len(results[0]) >= 1:
              annotated_frame = results[0].plot()
              image = np.array(annotated_frame)
              name = "/content/detections/{title}/frame_{frameNo}.jpg".format(title=title, frameNo=currentFrame)
              cv2.imwrite(name, image)

          currentFrame += 1

          # Break the loop if 'q' is pressed
          if cv2.waitKey(1) & 0xFF == ord("q"):
              break
      else:
          # Break the loop if the end of the video is reached
          os.chdir("/content")
          break

  # Release the video capture object and close the display window
  cap.release()



Streaming output truncated to the last 5000 lines.
0: 288x480 1 drone, 18.7ms
Speed: 1.7ms preprocess, 18.7ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 drone, 18.7ms
Speed: 2.0ms preprocess, 18.7ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 drone, 18.8ms
Speed: 1.8ms preprocess, 18.8ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 drone, 18.7ms
Speed: 1.9ms preprocess, 18.7ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 drone, 18.8ms
Speed: 1.8ms preprocess, 18.8ms inference, 1.3ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 drone, 18.9ms
Speed: 2.0ms preprocess, 18.9ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 drone, 18.8ms
Speed: 1.4ms preprocess, 18.8ms inference, 1.4ms postprocess per image at shape (1, 3, 288, 480)

0: 288x480 1 drone, 18.7ms
Speed: 1.5ms preprocess, 18.7ms inference, 1.4m